In [3]:
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelEncoder,StandardScaler,MinMaxScaler
import seaborn as sns
from sklearn.model_selection import train_test_split, StratifiedShuffleSplit
from sklearn.neighbors import KNeighborsClassifier
from sklearn import metrics
import missingno as msno #library to visualize missing data

In [60]:
df=pd.read_csv("loan.csv")

C:\ProgramData\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:2785: DtypeWarning: Columns (135,204,274,417) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [61]:
df.head()

id   f1  f2        f3    f4  f5     f6      f7      f8      f9  ...   f770  \
0   1  126  10  0.686842  1100   3  13699  7201.0  4949.0  126.75  ...      5   
1   2  121  10  0.782776  1100   3  84645   240.0  1625.0  123.52  ...      6   
2   3  126  10  0.500080  1100   3  83607  1800.0  1527.0  127.76  ...     13   
3   4  134  10  0.439874  1100   3  82642  7542.0  1730.0  132.94  ...      4   
4   5  109   9  0.502749  2900   4  79124    89.0   491.0  122.72  ...     26   

   f771  f772  f773    f774    f775  f776  f777  f778  loss  
0  2.14 -1.54  1.18  0.1833  0.7873     1     0     5     0  
1  0.54 -0.24  0.13  0.1926 -0.6787     1     0     5     0  
2  2.89 -1.73  1.04  0.2521  0.7258     1     0     5     0  
3  1.29 -0.89  0.66  0.2498  0.7119     1     0     5     0  
4  6.11 -3.82  2.51  0.2282 -0.5399     0     0     5     0  

[5 rows x 771 columns]

In [62]:
df.drop('id',inplace=True,axis=1)
df.head()

f1  f2        f3    f4  f5     f6      f7      f8      f9     f10  ...   \
0  126  10  0.686842  1100   3  13699  7201.0  4949.0  126.75  126.03  ...    
1  121  10  0.782776  1100   3  84645   240.0  1625.0  123.52  121.35  ...    
2  126  10  0.500080  1100   3  83607  1800.0  1527.0  127.76  126.49  ...    
3  134  10  0.439874  1100   3  82642  7542.0  1730.0  132.94  133.58  ...    
4  109   9  0.502749  2900   4  79124    89.0   491.0  122.72  112.77  ...    

   f770  f771  f772  f773    f774    f775  f776  f777  f778  loss  
0     5  2.14 -1.54  1.18  0.1833  0.7873     1     0     5     0  
1     6  0.54 -0.24  0.13  0.1926 -0.6787     1     0     5     0  
2    13  2.89 -1.73  1.04  0.2521  0.7258     1     0     5     0  
3     4  1.29 -0.89  0.66  0.2498  0.7119     1     0     5     0  
4    26  6.11 -3.82  2.51  0.2282 -0.5399     0     0     5     0  

[5 rows x 770 columns]

In [63]:
g = df.columns.to_series().groupby(df.dtypes).groups
datatypedict={k.name: v for k, v in g.items()}
datatypedict

{'int64': Index(['f1', 'f2', 'f4', 'f5', 'f6', 'f13', 'f16', 'f24', 'f25', 'f33', 'f34',
        'f35', 'f36', 'f37', 'f38', 'f68', 'f73', 'f74', 'f284', 'f285', 'f286',
        'f287', 'f293', 'f294', 'f295', 'f296', 'f301', 'f302', 'f303', 'f304',
        'f309', 'f310', 'f311', 'f312', 'f317', 'f318', 'f319', 'f320', 'f325',
        'f326', 'f327', 'f328', 'f335', 'f336', 'f337', 'f345', 'f354', 'f362',
        'f371', 'f379', 'f380', 'f381', 'f382', 'f400', 'f403', 'f408', 'f417',
        'f418', 'f427', 'f447', 'f465', 'f467', 'f477', 'f478', 'f480', 'f521',
        'f530', 'f531', 'f532', 'f597', 'f599', 'f656', 'f657', 'f658', 'f659',
        'f670', 'f674', 'f681', 'f693', 'f699', 'f700', 'f701', 'f702', 'f722',
        'f725', 'f728', 'f729', 'f730', 'f731', 'f732', 'f736', 'f741', 'f764',
        'f770', 'f776', 'f777', 'f778', 'loss'],
       dtype='object'),
 'float64': Index(['f3', 'f7', 'f8', 'f9', 'f10', 'f14', 'f15', 'f17', 'f18', 'f19',
        ...
        'f765', 'f76

In [64]:
categorical_features = datatypedict['object'].tolist()

In [65]:
continuous_features  = datatypedict['float64'].tolist()+datatypedict['int64'].tolist()

In [66]:
#Function to get the features containing null values
def get_null_count(df):
   
    na_count_df=df.isnull().sum().sort_values(ascending=False)/len(df)#columnwise ratio of null values
    na_count_df=na_count_df[na_count_df>0]#choose only the columns whic have null values

    return na_count_df
        

In [67]:
if df.isnull().values.any():
    #obtain the list of features containing null values
    null_features=get_null_count(df).index.values
    for i in null_features:
        if i in categorical_features:
            df[i].fillna('unavailable',inplace=True)
        else:
            df[i].fillna(df[i].mean(),inplace=True) 
    #df.fillna(median,inplace=True)
    

In [68]:
#Convert categorical features to numerical
def encoding(df,categorical_features):
    
    df_cat = df[categorical_features]
    #By defaault, convert all the categories to numerical forms using Label encoder
    le = {}
    for i in range(len(df_cat.columns.tolist())):
        le[i] = LabelEncoder()
        df_cat.iloc[:,i] = le[i].fit_transform(df_cat.iloc[:,i].astype(str))

    return df_cat


In [69]:
encoded_data  = encoding(df,categorical_features)
df          = pd.concat([encoded_data,df[continuous_features]],axis=1)
print('Encoded data looks like this: \n',df.head())

C:\ProgramData\Anaconda3\lib\site-packages\pandas\core\indexing.py:630: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item_labels[indexer[info_axis]]] = value
C:\ProgramData\Anaconda3\lib\site-packages\pandas\core\indexing.py:543: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s


Encoded data looks like this: 
    f137   f138   f206  f207  f276   f277  f338    f390   f391   f419  ...   \
0  5857  26247   7394  8190  4961  24084  6912    1661  14060   2053  ...    
1  1538   5011   9243  3180  5514  10194  7208   74509  76286  22731  ...    
2  1154  25611   5036  8222    86  24728  4416   43698  58273  10376  ...    
3  3574    824   2451  7612  1223  15024   651  101608  10695  15372  ...    
4  4254    604  10346  3561  2372  10883  1953   29846  65921   7865  ...    

       f731        f732  f736  f741  f764  f770  f776  f777  f778  loss  
0  12587508  1611786276     1     6     1     5     1     0     5     0  
1  13982483  1763226997     1     7     1     6     1     0     5     0  
2  29139977  3722679463     1    14     1    13     1     0     5     0  
3  11786291  1570641075     1     5     1     4     1     0     5     0  
4  55511449  7130863233     1    27     1    26     0     0     5     0  

[5 rows x 770 columns]


In [70]:
scaler = MinMaxScaler()
df=pd.DataFrame(scaler.fit_transform(df),columns=df.columns)

In [71]:
df.head()

f137      f138      f206      f207      f276      f277      f338  \
0  0.869507  0.842573  0.368117  0.564438  0.850652  0.838901  0.797968   
1  0.228325  0.160862  0.460171  0.219159  0.945473  0.355080  0.832140   
2  0.171318  0.822157  0.250722  0.566644  0.014746  0.861333  0.509813   
3  0.530582  0.026452  0.122025  0.524604  0.209705  0.523320  0.075156   
4  0.631532  0.019389  0.515085  0.245417  0.406722  0.379080  0.225468   

       f390      f391      f419  ...       f731      f732  f736      f741  \
0  0.015870  0.134342  0.087154  ...   0.026024  0.022826   0.0  0.017964   
1  0.711908  0.728907  0.964977  ...   0.030266  0.025789   0.0  0.023952   
2  0.417519  0.556795  0.440482  ...   0.076366  0.064120   0.0  0.065868   
3  0.970830  0.102190  0.652573  ...   0.023587  0.022021   0.0  0.011976   
4  0.285168  0.629871  0.333885  ...   0.156571  0.130791   0.0  0.143713   

   f764      f770  f776  f777      f778  loss  
0   0.0  0.018072   1.0   0.0  0.002479   0.0  
1   0.0  0.024096   1.0   0.0  0.002479   0.0  
2   0.0  0.066265   1.0   0.0  0.002479   0.0  
3   0.0  0.012048   1.0   0.0  0.002479   0.0  
4   0.0  0.144578   0.0   0.0  0.002479   0.0  

[5 rows x 770 columns]

In [72]:
X=df.iloc[:,:-1]
y=df.iloc[:,-1]


# Splitting the dataset into the Training set and Test set
from sklearn.cross_validation import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 0)

In [85]:
# Part 2 - Making the ANN

# Importing the Keras libraries and package
# Sequential module - initialize neural network
# Dense - layers of ANN
import keras
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras import regularizers
from keras import metrics
# Initialising the ANN
classifier = Sequential()

classifier.add(Dense(350, input_dim = 769, kernel_initializer = 'normal', activation = 'relu',
                     kernel_regularizer=regularizers.l2(0)))

#classifier.add(Dense(256, kernel_initializer = 'glorot_uniform', activation = 'relu' ,kernel_regularizer=regularizers.l2(0.001)))

#classifier.add(Dense(64, kernel_initializer = 'glorot_uniform', activation = 'relu' ,kernel_regularizer=regularizers.l2(0.1)))

classifier.add(Dense(1, kernel_initializer = 'normal'))

classifier.compile(optimizer = 'adam', loss = 'mean_squared_error', metrics = ['accuracy'])

classifier.fit(X_train, y_train,epochs = 50)


Epoch 1/50
84376/84376 [==============================] - 167s 2ms/step - loss: 0.0038 - acc: 0.9058
Epoch 2/50
84376/84376 [==============================] - 70s 830us/step - loss: 0.0020 - acc: 0.9070
Epoch 3/50
84376/84376 [==============================] - 71s 846us/step - loss: 0.0019 - acc: 0.9070
Epoch 4/50
84376/84376 [==============================] - 74s 876us/step - loss: 0.0019 - acc: 0.9070
Epoch 5/50
84376/84376 [==============================] - 73s 869us/step - loss: 0.0019 - acc: 0.9070
Epoch 6/50
84376/84376 [==============================] - 72s 853us/step - loss: 0.0019 - acc: 0.9070
Epoch 7/50
84376/84376 [==============================] - 76s 896us/step - loss: 0.0019 - acc: 0.9070
Epoch 8/50
84376/84376 [==============================] - 77s 911us/step - loss: 0.0019 - acc: 0.9070
Epoch 9/50
84376/84376 [==============================] - 73s 866us/step - loss: 0.0019 - acc: 0.9070
Epoch 10/50
84376/84376 [==============================] - 76s 896us/step - loss: 0

In [86]:
from sklearn.metrics import mean_squared_error
y_pred = classifier.predict(X_test)
loss = mean_squared_error( y_test, y_pred)

In [87]:
loss

0.0018815646323807507